In [54]:
import datetime
from decimal import Decimal, ROUND_HALF_UP
import pandas as pd

#### Für welches Jahr soll berechnet werden?

In [55]:
Zieljahr = 2021

In [61]:
def Gradtage_berechnen(Abre_Beginn, Montagetermin):
    # Juni bis August an "VMSG-Verbrauchsrechner Hochrechnung Grad- und Kalendertage.xlsx" orientiert, alternativ wäre 13/13,5/13,5 exakter
    res = 0
    Gradtage = {
        1: 170,
        2: 150,
        3: 130,
        4: 80,
        5: 40,
        6: 13,    
        7: 13.5,
        8: 13.5,
        9: 30,    
        10: 80,
        11: 120,
        12: 160    
    }
    
    # prüfen ob Monat anteilig oder komplett
    Abre_Beginn_Monat = Abre_Beginn.month
    while(Abre_Beginn_Monat != Montagetermin.month):
        res += Gradtage[Abre_Beginn_Monat]

        if(Abre_Beginn_Monat == 12): Abre_Beginn_Monat = 1
        else: Abre_Beginn_Monat += 1
        
    if(Montagetermin.day != 1):
        if(Montagetermin.month == 2 and not ist_schaltjahr(Montagetermin.year)):
            res += (Gradtage[Montagetermin.month]/28)*(Montagetermin.day-1)
        else:
            res += (Gradtage[Montagetermin.month]/Monatstage[Montagetermin.month])*(Montagetermin.day-1)

    res = Decimal(str(res))
    res = res.quantize(Decimal('1'), rounding=ROUND_HALF_UP)
    return(res, 1000-res)

def ist_schaltjahr(Jahr):
    if ((Jahr % 4 == 0 and Jahr % 100 != 0) or (Jahr % 400 == 0)):
        return(True)
    else:
        return(False)


Monatstage = {
    1: 31,
    2: 29,
    3: 31,
    4: 30,
    5: 31,
    6: 30,    
    7: 31,
    8: 31,
    9: 30,    
    10: 31,
    11: 30,
    12: 31     
}

data = []

for Jahr in range(Zieljahr-4,Zieljahr+1):
    for Monat in range(1,13):
        for Tag in range(1,32):
            Schaltjahr = ist_schaltjahr(Jahr) 
            
            if((Tag > Monatstage[Monat]) or (Monat == 2 and Tag == 29 and not Schaltjahr)):
                continue
            for Abre_Jahr in range(Zieljahr-4,Zieljahr+1):
                for Abre_Mon in range(1,13):
                    Abre_Beginn   = datetime.datetime.strptime(f'01.{Abre_Mon:02d}.{(Abre_Jahr)}', "%d.%m.%Y")
                    Abre_Ende     = datetime.datetime.strptime(f'01.{Abre_Mon:02d}.{(Abre_Jahr)+1}', "%d.%m.%Y") - datetime.timedelta(days=1)
                    Montagetermin = datetime.datetime.strptime(f'{Tag:02d}.{Monat:02d}.{(Jahr)}', "%d.%m.%Y")
                    
                    if (Abre_Beginn >= Montagetermin):
                        gradtage_res = [0,1000]
                    elif (Montagetermin > Abre_Ende):
                        gradtage_res = [1000,0]
                    else:
                        gradtage_res = Gradtage_berechnen(Abre_Beginn, Montagetermin)
                                                                    
                    Eintrag = {
                        "Abre_Beginn":         Abre_Beginn, 
                        "Abre_Ende" :          datetime.datetime.strptime(f'01.{Abre_Mon:02d}.{(Abre_Jahr)+1}', "%d.%m.%Y") - datetime.timedelta(days=1), 
                        "Montagetermin":       Montagetermin, 
                        "Gradtage_fehlen":     gradtage_res[0], 
                        "Gradtage_abgedeckt":  gradtage_res[1]                       
                    }
                    
                    data.append(Eintrag)
                    
df = pd.DataFrame(data, columns= ("Abre_Beginn", "Abre_Ende", "Montagetermin", "Gradtage_fehlen", "Gradtage_abgedeckt"))
df.to_csv(f'Gradtage-{datetime.date.today().strftime("%Y-%m-%d")}.csv', index=False, sep=';') #, compression="zip", encoding="iso-8859-15"

In [62]:
df

,Abre_Beginn,Abre_Ende,Montagetermin,Gradtage_fehlen,Gradtage_abgedeckt
0,2017-01-01,2017-12-31,2017-01-01,0,1000
1,2017-02-01,2018-01-31,2017-01-01,0,1000
2,2017-03-01,2018-02-28,2017-01-01,0,1000
3,2017-04-01,2018-03-31,2017-01-01,0,1000
4,2017-05-01,2018-04-30,2017-01-01,0,1000
...,...,...,...,...,...
109555,2021-08-01,2022-07-31,2021-12-31,398,602
109556,2021-09-01,2022-08-31,2021-12-31,385,615
109557,2021-10-01,2022-09-30,2021-12-31,355,645
109558,2021-11-01,2022-10-31,2021-12-31,275,725
